In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score

In [35]:
listings = pd.read_csv("./listings.csv")
listings = listings.iloc[:,12:]
rev = listings.columns
#print(rev)
temp = []
for r in rev:
    if "review" in r:
        temp.append(r)
temp.remove('number_of_reviews')
temp = ['price_log','city', 'state' ,'zipcode', 'country', 'has_availability', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'market', "monthly_price", "security_deposit", "weekly_price", "cleaning_fee", "calendar_updated", "calendar_last_scraped", "first_review", "last_review"] + temp
#print(temp)
listings.drop(temp, axis = 1, inplace=True)
listings['price'] = listings['price'].apply(lambda x: x.replace(",", ""))
listings['price'] = np.log(pd.to_numeric(listings['price']))
listings['extra_people'] = pd.to_numeric(listings['extra_people'])
listings['latitude'] = pd.to_numeric(listings['latitude'])
listings['longitude'] = pd.to_numeric(listings['longitude'])
listings = listings.fillna(listings.mean())
X = listings.drop('price', axis = 1)
Y = listings['price']
listings.head()

,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,...,number_of_reviews,cancellation_policy,TV,Kitchen,Internet,Pet Friendly,Heating,Air Conditioning,washer,dryer
0,42.282619,-71.133068,t,House,Entire home/apt,4,1.5,2.0,3.0,Real Bed,...,0,moderate,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
1,42.286241,-71.134374,t,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,...,36,moderate,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,42.292438,-71.135765,t,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,...,41,moderate,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
3,42.281106,-71.121021,f,House,Private room,4,1.0,1.0,2.0,Real Bed,...,1,moderate,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
4,42.284512,-71.136258,t,House,Private room,2,1.5,1.0,2.0,Real Bed,...,29,flexible,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [36]:
xx = ['property_type', 'room_type', "bed_type",
 "cancellation_policy","is_location_exact"]
X = pd.get_dummies(X, columns = xx)
#print(xx)
#print(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)

In [37]:
rf = RandomForestRegressor(oob_score = True, n_estimators = 1000, max_depth = 30, min_samples_split = 15, max_features = "sqrt")
print("\n Fitting Random Forest model...")
fit = rf.fit(X_train, Y_train)
print("\n Making Random Forest predictions...")
predictions = rf.predict(X_test)
print()
print("RMSE: ", mean_squared_error(Y_test, predictions), "\n")
print("R^2: ", r2_score(Y_test, predictions), "\n")


 Fitting Random Forest model...

 Making Random Forest predictions...
()
('RMSE: ', 0.12849719116672337, '\n')
('R^2: ', 0.68829815403138861, '\n')


feature_names = X_train.columns
importances = rf.feature_importances_
feats = {}
figComp= plt.figure(figsize=(50,20))
for feature, importance in zip(feature_names, importances):
    feats[feature] = importance #add the name/value pair 
feature_importance_df = pd.DataFrame.from_dict(feats, orient = 'index').rename(columns = {0: 'Importance'})
feature_importance_df = feature_importance_df.sort_values(by = 'Importance')
feature_importance_df.plot(kind = 'bar')
plt.savefig("feature_importance.png")
plt.show()